<a href="https://www.kaggle.com/code/cmdhmn/s5e12-cmd-hmn-lightgbm?scriptVersionId=283531910" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import gc
import warnings
import pickle
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

In [ ]:
class CONFIG:
    SEED = 67
    TARGET = 'diagnosed_diabetes'
    TEST_SIZE = 0.2

    TRAIN_PATH = '/kaggle/input/playground-series-s5e12/train.csv'
    TEST_PATH = '/kaggle/input/playground-series-s5e12/test.csv'

    FOLDS = 10

cfg = CONFIG()

In [ ]:
def print_design(data: None| str = None):
    print('=' * 25)
    if data:
        print(data)
        print('=' * 25)
        print()

def load_pickle(path):
    try:
        with open(path, 'rb') as f:
            data =  pickle.load(f)
        return data
    except:
        print('Please provide a valid path or filename.')


def write_pickle(path, data):
    try:
        with open(path, 'wb') as f:
            pickle.dump(f, data)
    except:
        print('Please provide a valid path or filename.')

In [ ]:
def import_data(cfg: CONFIG = cfg, sep_ids=True):
    print_design("1. IMPORTING DATASET")
    train = pd.read_csv(cfg.TRAIN_PATH)
    test = pd.read_csv(cfg.TEST_PATH)
    print('\n- Successfully imported the datasets\n')

    test_ids = None
    if sep_ids:
        print('--- Separating and Removing Ids ---\n')
        train = train.drop('id', axis=1)
        test_ids = test.id
        test = test.drop('id', axis=1)
        print('- Successfully done!')
        return train, test, test_ids

    print_design()
        
    return train, test

In [ ]:
def object_to_category(train, test, return_cat_cols=True):
    print_design('2. Handling CATS')

    cat_cols = train.select_dtypes(['object']).columns

    for cat in cat_cols:
        print(f'- Transforming {cat}')
        train[cat] = train[cat].astype('category')
        test[cat] = test[cat].astype('category')

    print('\n- Successfully Completed')
    print_design()
    if return_cat_cols:
        return train, test, cat_cols
    return train, test

In [ ]:
def split_data(train, train_test=False):
    print_design("3. SPLIT DATA")

    X = train.drop([cfg.TARGET], axis=1)
    y = train[cfg.TARGET]

    print('\n- Successfully splitted\n')
    print_design()
    
    if train_test:
        return train_test_split(X, y, test_size=cfg.TEST_SIZE, random_state=cfg.SEED)

    return X, y

In [ ]:
def submit_test(test_ids, test_preds):
    print_design("5. SUBMITTING TEST")
    output = pd.DataFrame({
        'id': test_ids,
        cfg.TARGET: test_preds
    })

    output.to_csv('submission.csv', index=False)
    return output.head()

In [ ]:
def plot_train_val(train, val, title):
    assert len(train) == len(val), "Train and val lenght should be equal"

    plt.figure(figsize=(8, 4.5))

    plt.plot(train, label="Train Acc")
    plt.plot(val, label="Val Acc")

    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title(title)
    plt.legend()

    plt.grid(False)
    plt.tight_layout()
    plt.show()

In [ ]:
def train_model(X, y, test, model, cat_cols, plots=True):
    name = model['name']
    print_design(f'4. TRAINING MODEL {name}')

    skf = StratifiedKFold(n_splits=cfg.FOLDS, shuffle=True, random_state=cfg.SEED)

    oof_preds = np.zeros(X.shape[0])
    test_preds = np.zeros(test.shape[0])
    train_errors = []
    val_errors = []
    cv_folds = {}

    for fold, (tr, va) in enumerate(skf.split(X, y), 1):
        print(f'\n-- Fold {fold} --\n')

        X_train = X.iloc[tr]
        y_train = y[tr]
        X_val = X.iloc[va]
        y_val = y[va]

        if name == 'LIGHTGBM':
            lgb_dataset = lgb.Dataset(X_train, y_train, categorical_feature=cat_cols)
            tr_model = lgb.train(train_set=lgb_dataset, params=model['params'])


        train_preds = tr_model.predict(X_train)
        train_error = roc_auc_score(y_train, train_preds)
        
        val_error = roc_auc_score(y_val, tr_model.predict(X_val))

        train_errors.append(train_error)
        val_errors.append(val_error)
        oof_preds[tr] = train_preds

        test_preds += tr_model.predict(test)
        
        cv_folds[fold] = tr_model
        
        print(f'- Train Error {train_error:.6f}')
        print(f'- Val Error {val_error:.6f}\n')

    test_preds = test_preds/(skf.get_n_splits())

    print_design('MEAN OF ERRORS')
    print(f'- Train Error {np.mean(train_error):.6f}')
    print(f'- Val Error {np.mean(val_error):.6f}\n')

    data = {
        'name': name,
        'cv_models': cv_folds,
        'oof_preds': oof_preds,
        'test_preds': test_preds,
        'train_error': train_error,
        'val_error': val_error
    }

    write_pickle(f'{name}_{np.mean(val_error):.6f}_data.pkl', data)
    
    del tr_model, skf, X, y, X_train, X_val, y_train, y_val, train_preds
    gc.collect()

    if plots:
        print(f'\n-- Seeing some plots --\n')
    
        plot_train_val(train_errors, val_errors, name)
    
        print(" " * 3)
    
    return oof_preds, test_preds

In [ ]:
models = {
    0: {
        "name": "LIGHTGBM",
        "params": {
               'objective': 'binary',
                'metric': 'auc',
                'learning_rate': 0.09990567366337798,
                'num_leaves':  203,      
                'n_estimators': 1000, 
                'max_depth': 12,
                'min_child_samples': 184,
                'colsample_bytree': 0.8,
                'lambda_l1': 0.3982350299843493, 
                'lambda_l2': 4.412853547060553e-05,
                'feature_fraction': 0.4099195570257677, 
                'bagging_fraction': 0.9994596878397021, 
                'bagging_freq': 1, 
                'subsample': 0.7,
                'random_state': cfg.SEED,
                'n_jobs': -1,
                'verbosity': -1   
        }
    }
}

In [ ]:
train, test, test_ids = import_data()

In [ ]:
train, test, cat_cols = object_to_category(train, test)

In [ ]:
X, y = split_data(train)

In [ ]:
oof_preds, test_preds = train_model(X, y, test, models[0], list(cat_cols))

In [ ]:
submit_test(test_ids, test_preds)